In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import geopandas as gpd

In [ ]:
df = pd.read_csv('Data/declarations-exterminations-punaises-de-lit.csv')

In [ ]:
df['DATE_DECLARATION'] = pd.to_datetime(df['DATE_DECLARATION'])
df['DATE_DEBUTTRAIT'] = pd.to_datetime(df['DATE_DEBUTTRAIT'])
df['DATE_FINTRAIT'] = pd.to_datetime(df['DATE_FINTRAIT'])
df['DATE_INSP_VISPRE'] = pd.to_datetime(df['DATE_INSP_VISPRE'])

df['LEN_TRAIT'] = df['DATE_FINTRAIT'] - df['DATE_DEBUTTRAIT'] 
df['LEN_TRAIT'] = df['LEN_TRAIT'].dt.days
df['NUM_WEEKS_TRAIT'] = np.ceil(df['LEN_TRAIT']/7)

In [ ]:
df_bb = df[df['NUM_WEEKS_TRAIT']<=52]

The date of the event is the minimum of date declaration, date inspection and date beginning of treatment, where the date that is kept is the one where at least 2 events occurred not so far apart. 

In [ ]:
len(df_bb)

In [ ]:
condition = ((df_bb['DATE_DEBUTTRAIT'] - df_bb['DATE_DECLARATION']).dt.days > -365) & ((df_bb['DATE_DEBUTTRAIT'] - df_bb['DATE_INSP_VISPRE']).dt.days > -365)

In [ ]:
condition2 = ((df_bb['DATE_DEBUTTRAIT'] - df_bb['DATE_DECLARATION']).dt.days < 365) & ((df_bb['DATE_DEBUTTRAIT'] - df_bb['DATE_INSP_VISPRE']).dt.days < 365)

In [ ]:
df_bb = df_bb[condition2]

In [ ]:
df_bb = df_bb[condition]

In [ ]:
len(df_bb)

In [ ]:
df_bb['DATE_EVENT'] = df_bb[['DATE_DECLARATION','DATE_INSP_VISPRE','DATE_DEBUTTRAIT']].min(axis = 1)

df_bb['MONTH_EVENT'] = df_bb['DATE_EVENT'].dt.month
event_week_tmp = []
event_year_tmp = []
for event in df_bb['DATE_EVENT']:
    event_week_tmp.append(event.isocalendar()[1])
    event_year_tmp.append(event.isocalendar()[0])    
df_bb['YEAR_EVENT'] = event_year_tmp
df_bb['WEEK_EVENT'] = event_week_tmp

In [ ]:
df_bb.columns

In [ ]:
df_bb_ = df_bb[[ 'NO_DECLARATION', 'No_QR', 'COORD_X', 'COORD_Y', 'NBR_EXTERMIN', 'DATE_EVENT', 'YEAR_EVENT',
       'MONTH_EVENT', 'WEEK_EVENT', 'LEN_TRAIT', 'NUM_WEEKS_TRAIT', 'NOM_QR',
       'NOM_ARROND', 'DATE_DECLARATION', 'DATE_INSP_VISPRE', 'DATE_DEBUTTRAIT']]

In [ ]:
df_bb_['YEAR_EVENT'].min()

In [ ]:
df_bb_['DATE_EVENT'].isna().value_counts()

In [ ]:
from shapely.geometry import Point
import shapely

In [ ]:
shapefiles = gpd.read_file('./quartierreferencehabitation/QuartierReferenceHabitation.shp', encoding = 'utf-8')

In [ ]:
shapefiles = shapefiles[shapefiles['nom_mun'] == 'Montréal']

In [ ]:
shapefiles['CENTROID'] = shapefiles.centroid

In [ ]:
shapefiles['CENTROID_X'] =shapefiles.centroid.x
shapefiles['CENTROID_Y'] =shapefiles.centroid.y


In [ ]:
len(shapefiles['CENTROID_X'].unique())

In [ ]:
df_ = df_bb_.merge(shapefiles[['no_qr', 'CENTROID_X', 'CENTROID_Y']], left_on = 'No_QR', right_on = 'no_qr')

In [ ]:
df_.to_csv('bbugs_clean.csv', index=False)

In [ ]:
df_.columns